In [1]:
import pandas as pd

In [8]:
df = pd.read_csv('./entityRelations/task2_train.csv')

In [4]:
import re
import sys
import numpy as np
from gensim.test.utils import get_tmpfile
from gensim.models import KeyedVectors
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.callbacks import ModelCheckpoint
from keras.models import Model
from keras.callbacks import History 
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Embedding, Input,BatchNormalization, Dense, Bidirectional,LSTM,Dropout,Activation, concatenate
from keras import backend as K

In [5]:
labelMapping = {'Component-Whole(e2,e1)': 0, 'Other': 1, 'Instrument-Agency(e2,e1)': 2, 'Member-Collection(e1,e2)': 3, 'Cause-Effect(e2,e1)': 4, 'Entity-Destination(e1,e2)': 5, 'Content-Container(e1,e2)': 6, 'Message-Topic(e1,e2)': 7, 'Product-Producer(e2,e1)': 8, 'Member-Collection(e2,e1)': 9, 'Entity-Origin(e1,e2)': 10, 'Cause-Effect(e1,e2)': 11, 'Component-Whole(e1,e2)': 12, 'Message-Topic(e2,e1)': 13, 'Product-Producer(e1,e2)': 14, 'Entity-Origin(e2,e1)': 15, 'Content-Container(e2,e1)': 16, 'Instrument-Agency(e1,e2)': 17, 'Entity-Destination(e2,e1)': 18}

In [11]:
df2 = pd.read_csv('./entityRelations/train_dataset.csv')
df2.head(5)

,Unnamed: 0,text,entity_indices,entities,labels,relations,edges
0,0,The system as described above has its greatest...,"(12, 15)","['configuration', 'elements']","Component-Whole(e2,e1)",['Component-Whole'],"(e2,e1)"
1,1,The child was carefully wrapped and bound into...,"(1, 9)","['child', 'cradle']",Other,['Other'],NaN
2,2,The author of a keygen uses a disassembler to ...,"(1, 7)","['author', 'disassembler']","Instrument-Agency(e2,e1)",['Instrument-Agency'],"(e2,e1)"
3,3,A misty ridge uprises from the surge,"(2, 6)","['ridge', 'surge']",Other,['Other'],NaN
4,4,The student association is the voice of the un...,"(1, 2)","['student', 'association']","Member-Collection(e1,e2)",['Member-Collection'],"(e1,e2)"


In [7]:
def dense_to_one_hot(labels_dense, num_classes):
    num_labels = labels_dense.shape[0]
    index_offset = np.arange(num_labels) * num_classes
    labels_one_hot = np.zeros((num_labels, num_classes))
    labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
    return labels_one_hot

In [29]:
num_classes = 19
train_text = []
from ast import literal_eval as make_tuple
p1 = [make_tuple(x)[0] for x in df2['entity_indices']]
p2 = [make_tuple(x)[1] for x in df2['entity_indices']]

In [26]:
p1 = [make_tuple(x)[0] for x in df2['entity_indices']]

In [30]:
for i in range(len(df)):
    split_sentence = df.iloc[i]['text'].split(' ')
    prune_text = " ".join(split_sentence[p1[i]:p2[i]+1])
    train_text.append(prune_text)   

In [32]:
len(train_text)

8000

In [33]:
tokenizer = Tokenizer(num_words=25000,lower=True,split=' ',char_level=False)
tokenizer.fit_on_texts(train_text)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 9889 unique tokens.


In [34]:
train_sentence_seq = tokenizer.texts_to_sequences(train_text)

In [35]:
train_sentence_seq

[[1542, 2, 971, 972],
 [346, 7, 683, 2856, 14, 4427, 6, 1, 801],
 [70, 2, 3, 4428, 76, 3, 4429],
 [4430, 2026, 4, 1, 4431],
 [128, 590],
 [1543, 18, 9, 4432, 1207, 444],
 [347, 5, 1, 2857, 42, 2, 1, 348, 12, 8, 2858, 4433, 143],
 [43, 20, 17, 252, 278, 6, 4434],
 [4435, 7, 30, 5, 3, 4436, 1544],
 [2859, 7, 4437, 6, 3, 2027, 2028, 129, 591],
 [973, 974, 5, 27, 802],
 [349, 13, 1545, 6, 2860],
 [2861, 1546, 1, 1208],
 [2029, 13, 17, 12, 8, 31, 684, 225],
 [1547, 20, 71, 6, 113, 2030, 2031],
 [510, 11, 2862, 685, 54, 1, 803],
 [2863, 2864, 4438, 4439, 5, 4440, 1209],
 [78, 99, 20, 226, 975, 4441, 2865, 4442, 4443, 4444, 2032],
 [114, 2866],
 [1548, 2, 4445, 1549],
 [1550,
  2,
  1,
  178,
  14,
  4446,
  2867,
  2,
  1,
  4447,
  4448,
  7,
  976,
  4,
  1,
  686,
  4449,
  4450],
 [2868, 2, 59, 179, 2869, 1551],
 [1210, 28, 1211, 130, 2, 1, 4451, 1552, 92, 12, 3, 2033],
 [973, 4452, 1553, 11, 4453, 977, 305],
 [1212, 2, 4454, 4455],
 [1554, 9, 24, 10, 3, 4456],
 [2870, 9, 50, 2, 1, 592, 

In [36]:
max_length = np.max([len(i) for i in train_sentence_seq])
print("max length:", max_length)

max length: 33


In [72]:
x_train_seq = sequence.pad_sequences(train_sentence_seq, maxlen=max_length)

In [73]:
x_train_seq

array([[   0,    0,    0, ...,    2,  971,  972],
       [   0,    0,    0, ...,    6,    1,  801],
       [   0,    0,    0, ...,   76,    3, 4429],
       ...,
       [   0,    0,    0, ...,    5,    1,  244],
       [   0,    0,    0, ...,    0,  680,  752],
       [   0,    0,    0, ...,    3,   80,  461]])

In [61]:
dependency_list = df['deps']

In [62]:
a, b = np.unique(dependency_list, return_counts=True)
a_sorted = a[np.argsort(b)[::-1]]
major_dep = a_sorted[:33]
major_dep

array(['nsubjpobj', 'nsubjpasspobj', 'dobjpobj', 'nsubjdobj', 'pobjpobj',
       'compoundpobj', 'attrpobj', 'compoundnsubj', 'conjpobj',
       'pobjdobj', 'nsubjcompound', 'compounddobj', 'dobjdobj',
       'nsubjnsubj', 'nsubjpunct', 'ROOTpobj', 'pobjcompound', 'attrdobj',
       'nsubjpasscompound', 'dobjcompound', 'nsubjconj',
       'compoundcompound', 'pobjnsubj', 'nsubjpassdobj', 'pobjconj',
       'nsubjpasspunct', 'nsubjamod', 'compoundnsubjpass', 'punctprep',
       'compoundattr', 'conjdobj', 'appospobj', 'pobjattr'], dtype=object)

In [63]:
dependency_list_filter = []
for dep in dependency_list:
    if dep in major_dep:
        dependency_list_filter.append(dep)
    else:
        dependency_list_filter.append("other")
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
le = LabelEncoder()
enc = OneHotEncoder(sparse=False)
lb = LabelBinarizer()

In [54]:
import numpy as np

def loadGloveModel(File):
    print("Loading Glove Model")
    f = open(File,'r',encoding='utf-8')
    gloveModel = {}
    for line in f:
        splitLines = line.split()
        word = splitLines[0]
        wordEmbedding = np.array([float(value) for value in splitLines[1:]])
        gloveModel[word] = wordEmbedding
    print(len(gloveModel)," words loaded!")
    return gloveModel

In [55]:
w2vModel = loadGloveModel('glove.42B.300d.txt')

Loading Glove Model
1917494  words loaded!


In [56]:
embedding_size = 300
num_words = len(word_index)+1
embedding_matrix = np.zeros((num_words, embedding_size))
oov = 0

In [57]:
for word, i in word_index.items():
    if word in w2vModel.keys():
        embedding_matrix[i] = w2vModel[word]
    else:
        oov+=1
print("OOV:",oov)        

OOV: 81


In [59]:
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Embedding, Input,InputLayer,BatchNormalization, Dense, Bidirectional,LSTM,Dropout,GRU,Activation
from keras.backend import sigmoid

In [60]:
def swish(x):
    return (sigmoid(x) * x)
get_custom_objects().update({'swish': Activation(swish)})

In [64]:
dependency_list_filter = lb.fit_transform(dependency_list_filter)

In [65]:
def train_BiLSTM(x_train, ancestor_train, dep_train, y_train, 
                 x_val, ancestor_val, dep_val, y_val,
                 embedding_matrix, max_length, max_features):
    embedding_size = 300
    batch_size = 64
    epochs = 50
    embedding_layer = Embedding(max_features,output_dim= embedding_size,
                            weights=[embedding_matrix],
                            input_length=max_length,
                            trainable=False)
    sequence_input = Input(shape=(max_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)
    lstm0 = Bidirectional(LSTM(256,activation="tanh",dropout=0.2,return_sequences = True,
                kernel_initializer='he_uniform'))(embedded_sequences)
    lstm1 = Bidirectional(LSTM(128,activation="tanh",dropout=0.2,return_sequences = True,
                kernel_initializer='he_uniform'))(lstm0)
    lstm2 = Bidirectional(LSTM(64,activation="tanh",dropout=0.2,return_sequences = False,
                kernel_initializer='he_uniform'))(lstm1)
    bn1 = BatchNormalization()(lstm2)
    
    # other feature inputs 
    ancestor_input = Input(shape=(2,))
    ancestor_feature = Dense(64, activation=swish)(ancestor_input)
    
    
    dep_input = Input(shape=(34,))
    dep_feature = Dense(128, activation=swish)(dep_input)
    
    combine_feature = concatenate([bn1, ancestor_feature, dep_feature])
    dense1 = Dense(64, activation=swish)(combine_feature)
    dropout1 = Dropout(0.5)(dense1)
    dense2 = Dense(32, activation=swish)(dropout1)
    dropout2 = Dropout(0.5)(dense2)
    preds = Dense(19, activation='softmax')(dropout2)
    model = Model([sequence_input, ancestor_input, dep_input], preds)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])
    filepath = "models/BiLSTM_3.hdf5" 
    checkpoint = ModelCheckpoint(filepath,monitor='val_acc',save_best_only=True)
    history = History()
    callbacks_list = [checkpoint, history]
    
    history = model.fit([x_train, ancestor_train, dep_train], y_train, 
                        validation_data=([x_val, ancestor_val, dep_val], y_val), 
                        epochs=epochs, 
                        batch_size=batch_size, 
                        callbacks=callbacks_list)
    return model, history

In [67]:
df_test = pd.read_csv('./entityRelations/task2_test.csv')
df2_test = pd.read_csv('./entityRelations/test_dataset.csv')


test_text = []

p1_test = [make_tuple(x)[0] for x in df2_test['entity_indices']]
p2_test = [make_tuple(x)[1] for x in df2_test['entity_indices']]

for i in range(len(df_test)):
    split_sentence = df_test.iloc[i]['text'].split(' ')
    prune_text = " ".join(split_sentence[p1_test[i]:p2_test[i]+1])
    test_text.append(prune_text) 
	
test_sentence_seq = tokenizer.texts_to_sequences(test_text)

x_test_seq = sequence.pad_sequences(test_sentence_seq, maxlen=max_length)

dependency_list_test = df_test['deps']

dependency_list_filter_test = []
for dep in dependency_list_test:
    if dep in major_dep:
        dependency_list_filter_test.append(dep)
    else:
        dependency_list_filter_test.append("other")
		
dependency_list_filter_test = lb.fit_transform(dependency_list_filter_test)

In [70]:
dependency_list_filter[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0])

In [71]:
train_mut_ancestors_list = df['ancestors']
test_mut_ancestors_list = df_test['ancestors']

In [75]:
train_label = [labelMapping[ele] for ele in df2['labels']]
train_label = dense_to_one_hot(np.array(train_label), 19)

In [76]:
test_label = [labelMapping[ele] for ele in df2_test['labels']]
test_label = dense_to_one_hot(np.array(test_label), 19)

In [77]:
x_test_seq = sequence.pad_sequences(test_sentence_seq, maxlen=max_length)

In [78]:
len(x_test_seq)

2717

In [79]:
model, history = train_BiLSTM(x_train_seq, train_mut_ancestors_list, dependency_list_filter, train_label, 
                     x_test_seq, test_mut_ancestors_list, dependency_list_filter_test, test_label,
                     embedding_matrix,
                     max_length,
                     len(word_index)+1)

Epoch 1/50


ValueError: in user code:

    c:\users\poorn\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    c:\users\poorn\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    c:\users\poorn\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    c:\users\poorn\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    c:\users\poorn\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replic

In [80]:
x_train_seq.shape()

TypeError: 'tuple' object is not callable

In [81]:
def _shuffle(X, feature1, feature2, y):
    randomize = np.arange(len(X))
    np.random.shuffle(randomize)
    return (X[randomize], feature1[randomize], feature2[randomize], y[randomize])

In [82]:
train_X, train_ancestor, train_dep, train_y = _shuffle(x_train_seq, 
                                                       train_mut_ancestors_list,
                                                       dependency_list_filter,
                                                       train_label)

In [83]:
model, history = train_BiLSTM(train_X, train_ancestor, train_dep, train_y, 
                     x_test_seq, test_mut_ancestors_list, dependency_list_filter_test, test_label,
                     embedding_matrix,
                     max_length,
                     len(word_index)+1)

Epoch 1/50


ValueError: in user code:

    c:\users\poorn\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    c:\users\poorn\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    c:\users\poorn\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    c:\users\poorn\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    c:\users\poorn\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replic

In [92]:
train_ancestor.shape

(8000,)

In [1]:
labelMappings = {'Other': 0,'Instrument-Agency(e1,e2)': 1, 'Instrument-Agency(e2,e1)': 2,
                     'Cause-Effect(e1,e2)': 3, 'Cause-Effect(e2,e1)': 4,'Member-Collection(e1,e2)': 5,
                     'Member-Collection(e2,e1)' : 6,'Entity-Destination(e1,e2)': 7, 'Entity-Destination(e2,e1)': 8,
                     'Content-Container(e1,e2)': 9, 'Content-Container(e2,e1)': 10,'Message-Topic(e1,e2)': 11,
                     'Message-Topic(e2,e1)': 12, 'Product-Producer(e1,e2)': 13,'Product-Producer(e2,e1)': 14,
                     'Entity-Origin(e1,e2)': 15, 'Entity-Origin(e2,e1)': 16,'Component-Whole(e1,e2)': 17,
                     'Component-Whole(e2,e1)': 18}

classToLabel = {0: ('Other','NA'),1: ('Instrument-Agency','(e1,e2)'), 2: ('Instrument-Agency','(e2,e1)'),
                    3: ('Cause-Effect','(e1,e2)'),4: ('Cause-Effect','(e2,e1)'),5: ('Member-Collection','(e1,e2)'),
                    6: ('Member-Collection','(e2,e1)'),7: ('Entity-Destination','(e1,e2)'),8: ('Entity-Destination','(e2,e1)'),
                    9: ('Content-Container','(e1,e2)'),10: ('Content-Container','(e2,e1)'),11: ('Message-Topic','(e1,e2)'),
                    12: ('Message-Topic','(e2,e1)'),13: ('Product-Producer','(e1,e2)'),14: ('Product-Producer','(e2,e1)'),
                    15: ('Entity-Origin','(e1,e2)'),16: ('Entity-Origin','(e2,e1)'),17: ('Component-Whole','(e1,e2)'),
                    18: ('Component-Whole','(e2,e1)')}

In [2]:
import json

In [5]:
json_object = json.dumps(labelMappings)
with open ('labelMappings.txt','w') as file:
    file.write(json_object)

In [7]:
json_object = json.dumps(classToLabel)
with open ('classToLabel.txt','w') as file:
    file.write(json_object)

In [8]:
with open('labelMappings.txt') as f: 
    data = f.read()
js = json.loads(data) 

In [9]:
js

{'Other': 0,
 'Instrument-Agency(e1,e2)': 1,
 'Instrument-Agency(e2,e1)': 2,
 'Cause-Effect(e1,e2)': 3,
 'Cause-Effect(e2,e1)': 4,
 'Member-Collection(e1,e2)': 5,
 'Member-Collection(e2,e1)': 6,
 'Entity-Destination(e1,e2)': 7,
 'Entity-Destination(e2,e1)': 8,
 'Content-Container(e1,e2)': 9,
 'Content-Container(e2,e1)': 10,
 'Message-Topic(e1,e2)': 11,
 'Message-Topic(e2,e1)': 12,
 'Product-Producer(e1,e2)': 13,
 'Product-Producer(e2,e1)': 14,
 'Entity-Origin(e1,e2)': 15,
 'Entity-Origin(e2,e1)': 16,
 'Component-Whole(e1,e2)': 17,
 'Component-Whole(e2,e1)': 18}